## General setup 

In [1]:
import os
import glob
import pandas as pd
import numpy as np
import pickle
import nltk
import graphlab
from graphlab import SFrame
from nltk.collocations import *       #Bad form
from nltk.corpus import stopwords
from nltk.tokenize import *
import string
import numpy as np
from nltk.stem.snowball import SnowballStemmer
LOCAL_DATA_PATH = 'C:\Users\Kevin OGallath34/political_history/data'
LOCAL_SAVE_PATH = 'C:\Users\Kevin OGallath34/political_history/processed_data'

## Read pickle file


In [2]:
#Read in Candidates_df pickle file
candidates_df = pd.read_pickle('C:\Users\Kevin OGallath34\political_history\processed_data\candidates_df1.pkl')
#print test_df.head()

In [3]:
candidates_df['affiliation']

index
3       Republican
6       Republican
7       Republican
8       Republican
9       Republican
10        Democrat
11      Republican
12      Republican
14      Republican
15      Republican
17        Democrat
18        Democrat
19        Democrat
20        Democrat
21        Democrat
22        Democrat
25        Democrat
27        Democrat
30      Republican
32      Republican
33        Democrat
34      Republican
36        Democrat
37      Republican
38        Democrat
39      Republican
40      Republican
41      Republican
42      Republican
44      Republican
           ...    
8023      Democrat
8024      Democrat
8026      Democrat
8027    Republican
8028      Democrat
8029    Republican
8030    Republican
8031    Republican
8032    Republican
8041      Democrat
8042      Democrat
8043      Democrat
8045      Democrat
8046      Democrat
8048      Democrat
8049      Democrat
8050      Democrat
8051      Democrat
8052      Democrat
8053      Democrat
8054      Democrat
8057  

In [4]:
#label data - 0 for democrat, 1 for republican
candidates_df['affiliation'].replace({'Democrat':0, 'Republican':1}, inplace=True)

In [5]:
#make new lists just with labels and text
labels = candidates_df['affiliation']
paragraph_text = candidates_df['text2']

##  Sentiment Analysis using TextBlob

In [6]:
from textblob import TextBlob as tb
from textblob_aptagger import PerceptronTagger
from textblob import Blobber

tb2 = Blobber(pos_tagger=PerceptronTagger())


## Determine Polarity of Subjective Sentences in Text (>0.49)

In [7]:
#  Load text2 into TextBlobber
#  textblob.sentences    parses text into each sentence
#  sentence.sentiment.subjectivity     Sentiment value for the sentence
#  sentence.sentiment.polarity          Polarity value for the sentence

def clean_text(text):
    tbs = tb2(text)
    pol = 0.0
    for sentence in tbs.sentences:
             sub = sentence.sentiment.subjectivity             # returns (polarity, subjectivity)
             if sub > 0.49:
                    pol1 = sentence.sentiment.polarity
                    pol = pol + pol1
    return(pol) 

# iterates through each Text field calculating the polarity value for the subjective sentences only

for _, text2 in candidates_df.iteritems():
     candidates_df['polarity'] =  candidates_df['text2'].apply(lambda row: clean_text(row))  

## Determine Polarity of All Sentences in Text


In [8]:
#  Load text2 into TextBlobber
#  textblob.sentences    parses text into each sentence
#  sentence.sentiment.subjectivity     Sentiment value for the sentence
#  sentence.sentiment.polarity          Polarity value for the sentence

def clean_text2(text):
    tbs = tb2(text)
    pol = 0.0
    for sentence in tbs.sentences:
             pol1 = sentence.sentiment.polarity             # returns (polarity, subjectivity)
             pol = pol + pol1
    return(pol) 

# iterates through each Text field calculating the polarity value for the subjective sentences only
for _, text2 in candidates_df.iteritems():
     candidates_df['polarity_all'] =  candidates_df['text2'].apply(lambda row: clean_text2(row))  

##   Finish cleaning up text

In [9]:
# load nltk's English stopwords as variable called 'stopwords'
stopwords = nltk.corpus.stopwords.words('english')

# load nltk's SnowballStemmer as variabled 'stemmer'
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")
;

''

In [10]:
from textblob import TextBlob as tb
from textblob_aptagger import PerceptronTagger
from textblob.taggers import NLTKTagger
from textblob import Blobber

nltk_tagger = NLTKTagger()
tb2 = Blobber(pos_tagger=nltk_tagger)

In [11]:
wordstring = []
def clean_tokens (text):
    tbs = tb2(text)
    words_para = tbs.words 
    return(words_para)
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    
for _, text2 in candidates_df.iteritems():
    candidates_df['words_para'] =  candidates_df['text2'].apply(lambda row: clean_tokens(row))

In [12]:
wordstring = []
for _, words_para in candidates_df.iteritems():
        words_list = candidates_df['words_para']

wordstring += words_list
word_all = wordstring.split()
word_freq = [word_all.count(w) for w in word_all] # a list comprehension word_tot = zip(word_all, word_freq)

TypeError: The `text` argument passed to `__init__(text)` must be a string, not <class 'textblob.blob.WordList'>

In [ ]:
def clean_stems(text):
    tbs = tb2(text)
    tokens = tbs.tags                                  
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    return(tokens)
                     
for _, text2 in candidates_df.iteritems():
     candidates_df['tokens'] =  candidates_df['text2'].apply(lambda row: clean_stems(row)) 

## alternative creation of tokens (compare results )

In [ ]:
# parse into sentences and tokenize into POS
def tag_pos(text):
    pos_tags=[]
    # first tokenize by sentence
    for sent in nltk.sent_tokenize(text): 
        tags   = nltk.pos_tag(sent)
        pos_tags = tags.append(tags)
        return(pos_tags)
# iterates through each Text field calculating the polarity value for the subjective sentences only
for _, text2 in candidates_df.iteritems():
     candidates_df['pos_tags'] =  candidates_df['text2'].apply(lambda row: tag_pos(row))

In [ ]:
# Filter POS Tokens to only Noun, Verb, Adjective, and Adverbs
for _, pos_tags in candidates_df.iteritems():
    candidates_df['pos_filter'] = candidates_df['pos_tags'].apply(lambda x: [(t[0],t[1]) for t in x if t[1]=='VB' or t[1]=='VBD' or t[1]=='VBG' or t[1]=='VBN' or t[1]=='VBP' or t[1]=='VBZ' or t[1]=='NN' or t[1]=='NNS' or t[1]=='NNP' or t[1]=='NNPS'
                                          or t[1]=='JJ' or t[1]=='JJR' or t[1]=='JJS' or t[1]=='RB' or t[1]=='RBR' or t[1]=='RBS']) 
   

## Save Dataframe as pickle (pos_df)

In [ ]:
candidates_df.keys()

In [ ]:
#export pos_df at this point for recovery
directory_name = LOCAL_SAVE_PATH
base_filename = 'pos_df'
suffix = '.pkl'
save_path = os.path.join(directory_name, base_filename + suffix)
candidates_df.to_pickle(save_path)

In [ ]:
import SFrame
sf = sframe.Sframe(data=candidates_df)
SFrame.show(sf)

In [ ]:
pos_final = candidates_df.loc[:,'date','speaker','affiliation','text2','pos_tags']

In [ ]:
#export pos_df at this point for recovery
directory_name = LOCAL_SAVE_PATH
base_filename = 'pos_final'
suffix = '.pkl'
save_path = os.path.join(directory_name, base_filename + suffix)
candidates_df.to_pickle(save_path)

##   Goverment vs Opposition impact on emotional polarity

In [ ]:
sum_df = candidates_df.loc[:,['affiliation','polarity','polarity_all']]


In [ ]:
grouped = sum_df.groupby('affiliation').mean()

In [ ]:
grouped

In [ ]:
import re
candidates_df['date'] = pd.to_datetime(candidates_df['date'])
candidates_df['year'] = candidates_df['date'].dt.year 
        
candidates_df['year']=candidates_df['year'].astype(str)   

#party of governing president when the debates occurred.    0 = democratic    1 = republican
presidents_dict = { 
"2000": 0,
"2001": 1,
"2002": 1,
"2003": 1,
"2004": 1,
"2005": 1,
"2006": 1,
"2007": 1,
"2008": 1, 
"2009": 0,
"2010": 0,
"2011": 0,
"2012": 0,
"2013": 0,
"2014": 0,
"2015": 0,
"2016": 0}
  
    

def expand_presidents(search_year):
     for year, opposition in presidents_dict.items():
        
        if year == search_year:
            return opposition


for _, year in candidates_df.iteritems():
     candidates_df['opposition'] =  candidates_df['year'].apply(lambda row: expand_presidents(row))  
    

In [ ]:
candidates_df['year']

In [ ]:
candidates_df['opposition']


## Visualization of Polarity totals

In [ ]:
subset_df = candidates_df.loc[:,['affiliation','polarity','polarity_all','opposition']
grouped = sum_df.groupby('affiliation','opposition').mean()

In [ ]:
import graphlab
from graphlab import SFrame
graphlab.canvas.set_target('ipynb')
sf = SFrame(data=candidates_df)
sf.show(sf)

In [ ]:
"""
Simple demo of a scatter plot.
"""
import numpy as np
import matplotlib.pyplot as plt



x = grouped
y = grouped
colors = np.random.rand(N)
area = np.pi * (15 * np.random.rand(N))**2  # 0 to 15 point radiuses

plt.scatter(x, y, s=area, c=colors, alpha=0.5)
plt.show()
x = np.random.rand(N)
y = np.random.rand(N)
colors = np.random.rand(N)
area = np.pi * (15 * np.random.rand(N))**2  # 0 to 15 point radiuses

plt.scatter(x, y, s=area, c=colors, alpha=0.5)


In [ ]:
import matplotlib.pylab as plt
candidates_df['affiliation'] == 1 
candidates_df['opposition'] == 1
colors = np.where(candidates_df['opposition'] == 1, 'k', 'r')
plt.scatter(candidates_df['opposition'], candidates_df['polarity'], s=120, c=colors)
plt.show()

In [ ]:
candidates_df['affiliation'] == 0 
candidates_df['opposition'] == 0
colors = np.where(candidates_df['opposition'] == 0, 'k', 'r')
plt.scatter(candidates_df['opposition'], candidates_df['polarity'], s=120, c=colors)
plt.show()
